In [55]:
# !pip install -U onnxruntime

In [1]:
import onnxruntime
import numpy as np
from sklearn.preprocessing import normalize

from tokenization_enc_dec_encn import EncDecTokenizer

In [2]:
tokenizer = EncDecTokenizer('./encn/vocab.txt')

In [3]:
ses = onnxruntime.InferenceSession('./cpm_2_0_encoder_onnx/__MODEL_PROTO.onnx')

In [4]:
%%time
outputs = ses.run(None, {'input_1': np.array([[5503, 145, 44, 44]], dtype=np.int32)})[0]

CPU times: user 40.2 s, sys: 12.8 s, total: 53 s
Wall time: 2.05 s


In [6]:
outputs[0, 0, :10]

array([ 0.5349248 ,  0.07466324,  0.00511339,  0.18261364,  0.00140795,
        0.02665715, -0.03043705, -0.01867004, -0.09950656, -0.16626081],
      dtype=float32)

In [7]:
outputs[0, 1, :10]

array([-0.00618988, -0.01400665, -0.03173439,  0.04005221, -0.00376131,
       -0.04000507, -0.01914792,  0.02522868,  0.04496328, -0.01402563],
      dtype=float32)

In [8]:
outputs[0, 2, :10]

array([-1.9101594e-01,  3.8479537e-01, -8.0629729e-02,  3.6000903e-04,
       -6.1258871e-02, -9.2276022e-02, -9.8567329e-02, -1.6662364e-01,
       -1.5630931e-02, -2.1819764e-01], dtype=float32)

In [6]:
input_text = '''当地时间9月6日是美国劳工节，但就在这一天，上千万美国劳动者却陷入新的困境。因为美国政府为疫情期间失业者提供的主要救助同日到期，而且白宫表示没有进一步延长救助的计划。
在德尔塔变异株已把美国推入新一轮疫情的背景下，失业救济的突然“断供”意味着有上千万美国人将全部或部分失去他们的生活来源。'''
input_ids = np.array([tokenizer.encode(input_text)], dtype=np.int32)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.061 seconds.
Prefix dict has been built successfully.


In [7]:
%%time
outputs = ses.run(None, {'input_1': input_ids})[0]

CPU times: user 2min 10s, sys: 0 ns, total: 2min 10s
Wall time: 3.96 s


In [60]:
def encode(text):
    token = [6] + tokenizer.encode(text)
    input_ids = np.array([token], dtype=np.int32)
    # input_ids = np.array([[tokenizer.encode(x)[0] for x in text]], dtype=np.int32)
    outputs = ses.run(None, {'input_1': input_ids})[0]
    # print(token, w)
    outputs = np.sum(outputs, axis=1)
    # outputs = np.mean(outputs, axis=1)
    # outputs = np.max(outputs, axis=1)
    # outputs = outputs[:, 0, :]
    outputs = normalize(outputs)
    return outputs

In [48]:
t = encode('添加提醒')

In [49]:
np.sum(encode('添加提醒') * encode('添加提醒-'))

0.6132251383207434

In [52]:
np.sum(encode('添加提醒') * encode('删除提醒'))

[6, 2596, 3085] [[[0.00105504]
  [0.45647969]
  [0.54246527]]]
[6, 3413, 3085] [[[0.00092251]
  [0.524754  ]
  [0.47432349]]]


0.7387952703761811

In [53]:
np.sum(encode('火锅') * encode('涮锅'))

[6, 7833] [[[7.65403750e-04]
  [9.99234596e-01]]]
[6, 18750, 2209] [[[2.86191271e-04]
  [8.94347722e-01]
  [1.05366086e-01]]]


0.6461303435438082

In [157]:
np.sum(encode('火锅') * encode('飞机'))

0.49493653

In [156]:
np.sum(encode('火锅') * encode('鸟'))

0.3924273

In [155]:
np.sum(encode('火锅') * encode('涮涮锅'))

0.48438495

In [154]:
np.sum(encode('成都') * encode('重庆'))

0.63463384

In [153]:
np.sum(encode('火锅') * encode('麻辣烫'))

0.24124652

In [152]:
np.sum(encode('星期一') * encode('星期二'))

0.76423216

In [163]:
np.sum(encode('添加提醒') * encode('增加提醒'))

0.66420686

In [164]:
np.sum(encode('添加提醒') * encode('添加一个提醒'))

0.66127133

In [150]:
np.sum(encode('添加提醒') * encode('删除提醒'))

0.6693349

In [162]:
np.sum(encode('给我的礼品') * encode('给我的礼物'))

0.7084629

In [186]:
np.sum(encode('添加提醒 ') * encode('添加提醒'))

0.51599